In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt

In [2]:
def selectkbest(indep_X,dep_Y,n):
    test= SelectKBest(score_func=chi2,k=n)
    fit1=test.fit(indep_X,dep_Y)
    selectk_features= fit1.transform(indep_X)
    return selectk_features

In [3]:
def split_scaler(indep_X,dep_Y):
    X_train,X_test,y_train,y_test=train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)
    sc=StandardScaler()
    X_train=sc.fit_transform(X_train)
    X_test=sc.transform(X_test)
    return X_train,X_test,y_train,y_test

In [5]:
def rscore_prediction(regressor,X_test,y_test):
    y_pred=regressor.predict(X_test)
    from sklearn.metrics import r2_score
    r_score=r2_score(y_test,y_pred)
    return regressor,r_score
def svm_linear(X_train,y_train,X_test,y_test):
    from sklearn.svm import SVR
    regressor = SVR(kernel="linear",C=3000)
    regressor.fit(X_train,y_train)
    regressor,r_score= rscore_prediction(regressor,X_test,y_test)
    return regressor,r_score
def svm_NL(X_train,y_train,X_test,y_test):
    from sklearn.svm import SVR
    regressor = SVR(C=100, epsilon=0.2)
    regressor.fit(X_train,y_train)
    regressor,r_score= rscore_prediction(regressor,X_test,y_test)
    return regressor,r_score
def Decision(X_train,y_train,X_test,y_test):
    from sklearn.tree import DecisionTreeRegressor
    regressor = DecisionTreeRegressor(random_state=0)
    regressor.fit(X_train,y_train)
    regressor,r_score= rscore_prediction(regressor,X_test,y_test)
    return regressor,r_score
def random(X_train,y_train,X_test,y_test):
    from sklearn.ensemble import RandomForestRegressor
    regressor = RandomForestRegressor(random_state=0)
    regressor.fit(X_train,y_train)
    regressor,r_score= rscore_prediction(regressor,X_test,y_test)
    return regressor,r_score

In [6]:
def selectk_regression(accsvml,accsvmnl,accdes,accrf):
    data = {'SVMl': accsvml, 'SVMnl': accsvmnl, 'Decision': accdes, 'Random': accrf}
    dataframe=pd.DataFrame(data, index=['ChiSquare'])
    return dataframe

In [7]:
dataset1=pd.read_csv("prepkidney_data.csv",index_col=None)


In [8]:
df2=pd.get_dummies(dataset1,drop_first=True)

In [9]:
indep_X=df2.drop("classification_ckd\t",1)
dep_Y= df2['classification_ckd\t']

In [10]:
kbest= selectkbest(indep_X,dep_Y,2)

accsvml=[]
accsvmnl=[]
accdes=[]
accrf=[]

In [11]:
X_train,X_test,y_train,y_test= split_scaler(kbest,dep_Y)


regressor,r_score= svm_linear(X_train,y_train,X_test,y_test)
accsvml.append(r_score)
regressor,r_score= svm_NL(X_train,y_train,X_test,y_test)
accsvmnl.append(r_score)
regressor,r_score= Decision(X_train,y_train,X_test,y_test)
accdes.append(r_score)
regressor,r_score= random(X_train,y_train,X_test,y_test)
accrf.append(r_score)

result= selectk_regression(accsvml,accsvmnl,accdes,accrf)

C:\Users\abina\Anac\envs\aiml\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\abina\Anac\envs\aiml\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\abina\Anac\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\abina\Anac\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


In [12]:
result

,SVMl,SVMnl,Decision,Random
ChiSquare,-0.820514,-3.644054,-0.010101,-0.020202
